In [ ]:
import torch
from torch import nn
import torchvision
from torchvision import transforms
from time import time


# 搭建神经网络
class myNet(nn.Module):
    def __init__(self):
        super(myNet,self).__init__()
        self.model=nn.Sequential(
            nn.Conv2d(3,32,5,1,2)
            ,nn.MaxPool2d(2)
            ,nn.Conv2d(32,32,5,1,2)
            ,nn.MaxPool2d(2)
            ,nn.Conv2d(32,64,5,1,2)
            ,nn.MaxPool2d(2)
            ,nn.Flatten() # 64x4x4
            ,nn.Linear(64*4*4,64)
            ,nn.Linear(64,10)
        )
    def forward(self,x):
        output=self.model(x)
        return output
    

    
if __name__=="__main__":
    mynet=myNet()
    input=torch.ones((64,3,32,32))
    output=mynet(input)
    print(output.shape)
    
# 上述文件保存为model.py

In [48]:
# 可以通过 from model import* 导入上述模型

import torch
from torch import nn
import torchvision
from torchvision import transforms
from torch.utils.tensorboard import SummaryWriter
from time import time

# 数据集
train_data=torchvision.datasets.CIFAR10('./data/05 data',train=True,transform=transforms.ToTensor(),download=True)
test_data=torchvision.datasets.CIFAR10('./data/05 data',train=False,transform=transforms.ToTensor(),download=True)

print("train_data's length: {}" .format(len(train_data)))
print("test_data's length: {}".format(len(test_data)))




# 利用dataloader加载数据
from torch.utils.data import DataLoader
train_data=DataLoader(train_data
                      ,batch_size=64 #每次抽取数据的大小
                      ,shuffle=False #是否打乱顺序
                     )
test_data=DataLoader(test_data
                     ,batch_size=64
                     ,shuffle=False)



# 搭建神经网络
class myNet(nn.Module):
    def __init__(self):
        super(myNet,self).__init__()
        self.model=nn.Sequential(
            nn.Conv2d(3,32,5,1,2)
            ,nn.MaxPool2d(2)
            ,nn.Conv2d(32,32,5,1,2)
            ,nn.MaxPool2d(2)
            ,nn.Conv2d(32,64,5,1,2)
            ,nn.MaxPool2d(2)
            ,nn.Flatten() # 64x4x4
            ,nn.Linear(64*4*4,64)
            ,nn.Linear(64,10)
        )
    def forward(self,x):
        output=self.model(x)
        return output
    

# 网络实例化
mynet=myNet()

# 损失
loss_fn=nn.CrossEntropyLoss()

# 优化器
learning_rate=0.01 #等价于1e-2
optimizer=torch.optim.SGD(mynet.parameters(),lr=learning_rate)

# 设置训练网络参数
total_train_step=0 # 训练与测试次数
total_test_step=0

epoch=3 # 训练轮数



# 使用tensorboard绘制图像
writer=SummaryWriter("myNet")

# 训练步骤
for i in range(epoch):
    print("..................train's round %d............"%(i+1))
    begin=time()
    mynet.train() #启动训练模式
    for data in train_data:
        imgs,targets=data
        outputs=mynet(imgs)
        loss=loss_fn(outputs,targets) #损失
        
        optimizer.zero_grad() #初始化梯度（调优）
        loss.backward() #反向传播
        optimizer.step()
        
        total_train_step+=1
        if total_train_step%100==0: #每1训练00次打印一次
            print(time()-begin)
            print("train round {},Loss {}".format(total_train_step,loss))
            writer.add_scalar("train_loss",loss.item(),total_train_step)
        
    
    # 测试步骤
    mynet.eval() # 启动调试模式
    total_test_loss=0 #测试的损失
    total_accuracy=0
    with torch.no_grad():
        for data in test_data:
            imgs,targets=data
            outputs=mynet(imgs)
            loss=loss_fn(outputs,targets) #损失
            total_test_loss+=loss.item() 
            accuracy=(outputs.argmax(1)==targets).sum() #预判正确
            total_accuracy+=accuracy
    print('total test accuracy {}'.format(total_accuracy/len(test_data))) #测试正确率
    print("total test loss {}".format(total_test_loss))
    writer.add_scalar("test_loss",total_test_loss,total_test_step)
    total_test_step+=1
    
    # 保存每次的模型
    torch.save(mynet,"./data/05 data/myNet/model_%d.pth"%i) #保存模型
    
writer.close()

Files already downloaded and verified
Files already downloaded and verified
train_data's length: 50000
test_data's length: 10000
..................train's round 1............
3.2972798347473145
train round 100,Loss 2.2900795936584473
6.563086271286011
train round 200,Loss 2.286755084991455
9.846892833709717
train round 300,Loss 2.2571284770965576
13.112164735794067
train round 400,Loss 2.192969799041748
16.374409675598145
train round 500,Loss 2.0804853439331055
19.62319827079773
train round 600,Loss 2.0272581577301025
22.89505910873413
train round 700,Loss 2.006061315536499
total test accuracy 17.66242027282715
total test loss 317.106058716774
..................train's round 2............
0.5906486511230469
train round 800,Loss 1.8972644805908203
3.8619191646575928
train round 900,Loss 1.8646212816238403
7.135700225830078
train round 1000,Loss 1.9697182178497314
10.456477403640747
train round 1100,Loss 1.9616525173187256
13.726745128631592
train round 1200,Loss 1.7277324199676514
17.00

### 获取正确率

In [38]:
import torch

outputs=torch.tensor([[0.1,0.2],[0.3,0.4]])
preds=outputs.argmax(1)
targets=torch.tensor([0,1])
print("accuracy: ",((preds==targets).sum()/len(targets)).item())
print(outputs.argmax(0))

accuracy:  0.5
tensor([1, 1])


### 使用GPU进行训练
##### 1.使用cuda ,在终端使用 nvidia-smi 查看cuda版本

##### 2.使用.to(divice),torch.divice("cuda")

In [49]:
# 可以通过 from model import* 导入上述模型
import torch
from torch import nn
import torchvision
from torchvision import transforms
from torch.utils.tensorboard import SummaryWriter
from time import time

print(torch.__version__,torch.cuda.is_available())


# 数据集
train_data=torchvision.datasets.CIFAR10('./data/05 data',train=True,transform=transforms.ToTensor(),download=True)
test_data=torchvision.datasets.CIFAR10('./data/05 data',train=False,transform=transforms.ToTensor(),download=True)

print("train_data's length: {}" .format(len(train_data)))
print("test_data's length: {}".format(len(test_data)))




# 利用dataloader加载数据
from torch.utils.data import DataLoader
train_data=DataLoader(train_data
                      ,batch_size=64 #每次抽取数据的大小
                      ,shuffle=False #是否打乱顺序
                     )
test_data=DataLoader(test_data
                     ,batch_size=64
                     ,shuffle=False)



# 搭建神经网络
class myNet(nn.Module):
    def __init__(self):
        super(myNet,self).__init__()
        self.model=nn.Sequential(
            nn.Conv2d(3,32,5,1,2)
            ,nn.MaxPool2d(2)
            ,nn.Conv2d(32,32,5,1,2)
            ,nn.MaxPool2d(2)
            ,nn.Conv2d(32,64,5,1,2)
            ,nn.MaxPool2d(2)
            ,nn.Flatten() # 64x4x4
            ,nn.Linear(64*4*4,64)
            ,nn.Linear(64,10)
        )
    def forward(self,x):
        output=self.model(x)
        return output
    

# 网络实例化
mynet=myNet()
if torch.cuda.is_available():
    mynet=mynet.cuda() #将模型转入cuda



# 损失
loss_fn=nn.CrossEntropyLoss()
loss_fn=loss_fn.cuda()



# 优化器
learning_rate=0.01 #等价于1e-2
optimizer=torch.optim.SGD(mynet.parameters(),lr=learning_rate)

# 设置训练网络参数
total_train_step=0 # 训练与测试次数
total_test_step=0

epoch=3 # 训练轮数



# 使用tensorboard绘制图像
writer=SummaryWriter("myNet")

# 训练步骤
for i in range(epoch):
    print("..................train's round %d............"%(i+1))
    begin=time()
    mynet.train() #启动训练模式
    for data in train_data:
        imgs,targets=data
        if torch.cuda.is_available():
            imgs=imgs.cuda()
            targets=targets.cuda()
        outputs=mynet(imgs)
        loss=loss_fn(outputs,targets) #损失
        
        optimizer.zero_grad() #初始化梯度（调优）
        loss.backward() #反向传播
        optimizer.step()
        
        total_train_step+=1
        if total_train_step%100==0: #每1训练00次打印一次
            print(time()-begin)
            print("train round {},Loss {}".format(total_train_step,loss))
            writer.add_scalar("train_loss",loss.item(),total_train_step)
        
    
    # 测试步骤
    mynet.eval() # 启动调试模式
    total_test_loss=0 #测试的损失
    total_accuracy=0
    with torch.no_grad():
        for data in test_data:
            imgs,targets=data
            if torch.cuda.is_available():
                imgs=imgs.cuda()
                targets=targets.cuda()
            outputs=mynet(imgs)
            loss=loss_fn(outputs,targets) #损失
            total_test_loss+=loss.item() 
            accuracy=(outputs.argmax(1)==targets).sum() #预判正确
            total_accuracy+=accuracy
    print('total test accuracy {}'.format(total_accuracy/len(test_data))) #测试正确率
    print("total test loss {}".format(total_test_loss))
    writer.add_scalar("test_loss",total_test_loss,total_test_step)
    total_test_step+=1
    
    # 保存每次的模型
    torch.save(mynet,"./data/05 data/myNet/model_%d.pth"%i) #保存模型
    
writer.close()

1.9.0+cpu False
Files already downloaded and verified
Files already downloaded and verified
train_data's length: 50000
test_data's length: 10000
..................train's round 1............
3.2937698364257812
train round 100,Loss 2.2912635803222656
6.592059850692749
train round 200,Loss 2.2880728244781494
9.889849185943604
train round 300,Loss 2.2815208435058594
13.225053310394287
train round 400,Loss 2.2367799282073975
16.52427053451538
train round 500,Loss 2.1794052124023438
19.754022359848022
train round 600,Loss 2.068948984146118
22.991310358047485
train round 700,Loss 2.0215542316436768
total test accuracy 17.503185272216797
total test loss 316.11104905605316
..................train's round 2............
0.5991342067718506
train round 800,Loss 1.9156901836395264
3.8938913345336914
train round 900,Loss 1.8651684522628784
7.201695203781128
train round 1000,Loss 1.9548271894454956
10.735470533370972
train round 1100,Loss 1.9753001928329468
14.244906663894653
train round 1200,Loss 1.

In [50]:
# 可以通过 from model import* 导入上述模型
import torch
from torch import nn
import torchvision
from torchvision import transforms
from torch.utils.tensorboard import SummaryWriter
from time import time

print(torch.__version__,torch.cuda.is_available())

device=torch.device("cpu") #设置设备

# 数据集
train_data=torchvision.datasets.CIFAR10('./data/05 data',train=True,transform=transforms.ToTensor(),download=True)
test_data=torchvision.datasets.CIFAR10('./data/05 data',train=False,transform=transforms.ToTensor(),download=True)

print("train_data's length: {}" .format(len(train_data)))
print("test_data's length: {}".format(len(test_data)))




# 利用dataloader加载数据
from torch.utils.data import DataLoader
train_data=DataLoader(train_data
                      ,batch_size=64 #每次抽取数据的大小
                      ,shuffle=False #是否打乱顺序
                     )
test_data=DataLoader(test_data
                     ,batch_size=64
                     ,shuffle=False)



# 搭建神经网络
class myNet(nn.Module):
    def __init__(self):
        super(myNet,self).__init__()
        self.model=nn.Sequential(
            nn.Conv2d(3,32,5,1,2)
            ,nn.MaxPool2d(2)
            ,nn.Conv2d(32,32,5,1,2)
            ,nn.MaxPool2d(2)
            ,nn.Conv2d(32,64,5,1,2)
            ,nn.MaxPool2d(2)
            ,nn.Flatten() # 64x4x4
            ,nn.Linear(64*4*4,64)
            ,nn.Linear(64,10)
        )
    def forward(self,x):
        output=self.model(x)
        return output
    

    
    
# 网络实例化
mynet=myNet()
mynet=mynet.to(device)




# 损失
loss_fn=nn.CrossEntropyLoss()
loss_fn=loss_fn.to(device)



# 优化器
learning_rate=0.01 #等价于1e-2
optimizer=torch.optim.SGD(mynet.parameters(),lr=learning_rate)

# 设置训练网络参数
total_train_step=0 # 训练与测试次数
total_test_step=0

epoch=3 # 训练轮数



# 使用tensorboard绘制图像
writer=SummaryWriter("myNet")

# 训练步骤
for i in range(epoch):
    print("..................train's round %d............"%(i+1))
    begin=time()
    mynet.train() #启动训练模式
    for data in train_data:
        imgs,targets=data
        
        imgs=imgs.to(device)
        targets=targets.to(device)
        
        outputs=mynet(imgs)
        loss=loss_fn(outputs,targets) #损失
        
        optimizer.zero_grad() #初始化梯度（调优）
        loss.backward() #反向传播
        optimizer.step()
        
        total_train_step+=1
        if total_train_step%100==0: #每1训练00次打印一次
            print(time()-begin)
            print("train round {},Loss {}".format(total_train_step,loss))
            writer.add_scalar("train_loss",loss.item(),total_train_step)
        
    
    # 测试步骤
    mynet.eval() # 启动调试模式
    total_test_loss=0 #测试的损失
    total_accuracy=0
    with torch.no_grad():
        for data in test_data:
            imgs,targets=data
            imgs=imgs.to(device)
            targets=targets.to(device)
            outputs=mynet(imgs)
            loss=loss_fn(outputs,targets) #损失
            total_test_loss+=loss.item() 
            accuracy=(outputs.argmax(1)==targets).sum() #预判正确
            total_accuracy+=accuracy
    print('total test accuracy {}'.format(total_accuracy/len(test_data))) #测试正确率
    print("total test loss {}".format(total_test_loss))
    writer.add_scalar("test_loss",total_test_loss,total_test_step)
    total_test_step+=1
    
    # 保存每次的模型
    torch.save(mynet,"./data/05 data/myNet/model_%d.pth"%i) #保存模型
    
writer.close()

1.9.0+cpu False
Files already downloaded and verified
Files already downloaded and verified
train_data's length: 50000
test_data's length: 10000
..................train's round 1............
3.278568744659424
train round 100,Loss 2.2900986671447754
6.5730438232421875
train round 200,Loss 2.282193183898926
9.857674837112427
train round 300,Loss 2.260314464569092
13.172558546066284
train round 400,Loss 2.193427562713623
16.393816709518433
train round 500,Loss 2.068385362625122
19.675100088119507
train round 600,Loss 2.017336368560791
22.94585633277893
train round 700,Loss 2.01222825050354
total test accuracy 18.356687545776367
total test loss 311.3315589427948
..................train's round 2............
0.6111371517181396
train round 800,Loss 1.850699782371521
3.884445905685425
train round 900,Loss 1.7955965995788574
7.13021993637085
train round 1000,Loss 1.8945701122283936
10.409478902816772
train round 1100,Loss 1.9629818201065063
13.663058757781982
train round 1200,Loss 1.6778714656

### 模型验证套路

In [83]:
import torch
from PIL import Image

image_path="./data/imgs/dog.png"
image=Image.open(image_path)
image=image.convert("RGB")
print(image)

from torchvision import transforms
transform=transforms.Compose([transforms.Resize((32,32)),transforms.ToTensor()])
image=transform(image)
print(image.shape)


# 搭建神经网络
class myNet(nn.Module):
    def __init__(self):
        super(myNet,self).__init__()
        self.model=nn.Sequential(
            nn.Conv2d(3,32,5,1,2)
            ,nn.MaxPool2d(2)
            ,nn.Conv2d(32,32,5,1,2)
            ,nn.MaxPool2d(2)
            ,nn.Conv2d(32,64,5,1,2)
            ,nn.MaxPool2d(2)
            ,nn.Flatten() # 64x4x4
            ,nn.Linear(64*4*4,64)
            ,nn.Linear(64,10)
        )
    def forward(self,x):
        output=self.model(x)
        return output
    
# 使用第一种方法加载模型
model=torch.load("./data/05 data/myNet/model_1.pth")
image=torch.reshape(image,(-1,3,32,32))

# 在加载模型如果在CPU上加载GPU训练的模型
model=torch.load("./data/05 data/myNet/model_1.pth",map_location=torch.device("cpu")) 

model.eval()
with torch.no_grad():
    output=model(image)
    
print(output)
print(output.argmax(1)) #打印训练出来的结果

<PIL.Image.Image image mode=RGB size=1920x1080 at 0x1D4128986A0>
torch.Size([3, 32, 32])
tensor([[-0.7215,  1.5888, -0.4383,  0.5519, -1.0079,  0.6133, -0.3114,  0.2954,
         -0.5119, -0.0376]])
tensor([1])
